In [1]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn
import json
import csv
from datetime import date, datetime, timedelta

import os
import requests
from bs4 import BeautifulSoup

#import scipy.io as sio
#
#from scipy import interp
#
#from sklearn.preprocessing import StandardScaler
#from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
#from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
#from sklearn.svm import SVC
#import xgboost as xgb
#from sklearn.model_selection import GridSearchCV
#from sklearn.model_selection import StratifiedKFold
#from sklearn.externals import joblib
#
#from sklearn.model_selection import train_test_split
#from sklearn.metrics import roc_curve, auc
#from sklearn.metrics import confusion_matrix
##import scikitplot as skplt

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Coordinates conversion API

In [2]:
#https://geodesy.geo.admin.ch/reframe/navref?format=json&easting=2649930&northing=1177380&altitude=NaN&input=lv95&output=etrf93-ed
#https://geodesy.geo.admin.ch/reframe/navref?format=json&easting=649930&northing=177380&altitude=NaN&input=lv03&output=etrf93-ed

def swiss_coordinates_to_wgs84(easting, northing, system="MN95"):
    """Transforms swiss MN03 or MN95 coordinates to standard GPS WGS84,
    using https://geodesy.geo.admin.ch/reframe/navref API.
    (interface: https://www.swisstopo.admin.ch/en/maps-data-online/calculation-services/navref.html).
    Coordinates (MN03 and MN95) satisfy: easting > northing, everywhere in Switzerland."""
    E=str(int(easting))
    N=str(int(northing))
    service_url = "https://geodesy.geo.admin.ch/reframe/navref"
    
    if system=="MN95":
        if(len(E)!=7 or len(N)!=7):
            raise(ValueError("Length of MN95 coordinates must be 7 each."))
        resp = requests.get(service_url+"?format=json&easting="+E+"&northing="+N+"&altitude=NaN&input=lv95&output=etrf93-ed").json()
    
    elif system=="MN95-short":
        if(len(E)!=6 or len(N)!=6):
            raise(ValueError("Length of 'MN95-short' coordinates must be 6 each."))
        resp = requests.get(service_url+"?format=json&easting=2"+E+"&northing=1"+N+"&altitude=NaN&input=lv95&output=etrf93-ed").json()
    
    elif system=="MN03":
        if(len(E)!=6 or len(N)!=6):
            raise(ValueError("Length of MN03 coordinates must be 6 each."))
        resp = requests.get(service_url+"?format=json&easting="+E+"&northing="+N+"&altitude=NaN&input=lv03&output=etrf93-ed").json()
    
    else:
        raise(ValueError("Accepted systems: 'MN03' 'MN95' 'MN95-short'"))
    
    result = {"longitude" : float(resp["easting"]),
             "latitude" : float(resp["northing"])}
    return(result)

In [3]:
resp=swiss_coordinates_to_wgs84(2649930,1177380)
resp

{'longitude': 8.092031204981819, 'latitude': 46.74574275510551}

# Webscraping stations infos

In [4]:
#req = requests.get("https://www.hydrodaten.admin.ch/en/current-situation-table-discharge-and-water-levels.html")
#soup = BeautifulSoup(req.text,'html.parser')
#stations_list = soup.find("table", class_="table table-bordered").find("tbody").find_all("tr")

In [5]:
req = requests.get("https://www.hydrodaten.admin.ch/en/2019.html")
req.encoding = "utf-8"
soup = BeautifulSoup(req.text,'html.parser')
Stations_list_soup = soup.find("div", class_="well search").find("select").find_all("option")

In [6]:
Stations_names = pd.DataFrame({"BAFU_ID":[int(station_soup["value"]) for station_soup in Stations_list_soup],
             "Name":[station_soup.string for station_soup in Stations_list_soup]})
Stations_names

,BAFU_ID,Name
0,2416,"Aabach - Hitzkirch, Richensee ( 2416 )"
1,2312,"Aach - Salmsach, Hungerbühl ( 2312 )"
2,2135,"Aare - Bern, Schönau ( 2135 )"
3,2019,Aare - Brienzwiler ( 2019 )
4,2016,Aare - Brugg ( 2016 )
...,...,...
347,2017,Zugersee - Zug ( 2017 )
348,6723,Zürich - Aussersihl/Zweierplatz ( 6723 )
349,6600,Zürich - Schlachthof ( 6600 )
350,2014,Zürichsee (Obersee) - Schmerikon ( 2014 )


In [7]:
def webscrape_station(station_id):
    id_str = str(station_id)
    while(len(id_str)<4):
        id_str = "0"+id_str
    try:
        req = requests.get("https://www.hydrodaten.admin.ch/en/"+id_str+".html")
        req.encoding = "utf-8"
        soup = BeautifulSoup(req.text,'html.parser')
        station_info_soup = soup.find_all(class_="col-md-6")[-1].table.tbody.find_all("tr")
        infos = [info_.td.string for info_ in station_info_soup]
        
        gps03 = swiss_coordinates_to_wgs84(easting=int(infos[4][0:6]),northing=int(infos[4][9:15]),
                                         system="MN03")
        gps95 = swiss_coordinates_to_wgs84(easting=int(infos[4][0:6]),northing=int(infos[4][9:15]),
                                         system="MN95-short")
        data=pd.Series({"Elevation":float(infos[0][:-8]),
                        "Surface_catchment":float(infos[1][:-3]),
                        "Mean_elevation_catchment":float(infos[2][:-8]),
                        "Glaciation_percent":float(infos[3][:-1]),
                        "easting":int(infos[4][0:6]),
                        "northing":int(infos[4][9:15]),
                        "longitude03":gps03["longitude"],
                        "latitude03":gps03["latitude"],
                        "longitude95":gps95["longitude"],
                        "latitude95":gps95["latitude"]})
    except:
        data=pd.Series({"Elevation":np.NaN,
                        "Surface_catchment":np.NaN,
                        "Mean_elevation_catchment":np.NaN,
                        "Glaciation_percent":np.NaN,
                        "easting":np.NaN,
                        "northing":np.NaN,
                        "longitude03":np.NaN,
                        "latitude03":np.NaN,
                        "longitude95":np.NaN,
                        "latitude95":np.NaN})
    return(data)

In [8]:
Stations_infos = Stations_names.BAFU_ID.apply(webscrape_station)
Stations_BAFU = Stations_names.join(Stations_infos)
Stations_BAFU.to_csv("data_wrangled/Webscraped_BAFU_stations_info.csv",index=False)
Stations_BAFU.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 352 entries, 0 to 351
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   BAFU_ID                   352 non-null    int64  
 1   Name                      352 non-null    object 
 2   Elevation                 235 non-null    float64
 3   Surface_catchment         235 non-null    float64
 4   Mean_elevation_catchment  235 non-null    float64
 5   Glaciation_percent        235 non-null    float64
 6   easting                   235 non-null    float64
 7   northing                  235 non-null    float64
 8   longitude03               235 non-null    float64
 9   latitude03                235 non-null    float64
 10  longitude95               235 non-null    float64
 11  latitude95                235 non-null    float64
dtypes: float64(10), int64(1), object(1)
memory usage: 33.1+ KB


In [9]:
Stations_BAFU = pd.read_csv("data_wrangled/Webscraped_BAFU_stations_info.csv")
Stations_BAFU

,BAFU_ID,Name,Elevation,Surface_catchment,Mean_elevation_catchment,Glaciation_percent,easting,northing,longitude03,latitude03,longitude95,latitude95
0,2416,"Aabach - Hitzkirch, Richensee ( 2416 )",463.0,73.3,581.0,0.0,661390.0,230220.0,8.249139,47.220062,8.249129,47.220062
1,2312,"Aach - Salmsach, Hungerbühl ( 2312 )",406.0,47.4,467.0,0.0,744410.0,268400.0,9.357226,47.550427,9.357217,47.550432
2,2135,"Aare - Bern, Schönau ( 2135 )",502.0,2941.0,1596.0,5.8,600710.0,198000.0,7.447957,46.933093,7.447956,46.933092
3,2019,Aare - Brienzwiler ( 2019 ),570.0,555.0,2135.0,15.5,649930.0,177380.0,8.092037,46.745741,8.092031,46.745743
4,2016,Aare - Brugg ( 2016 ),332.0,11681.0,1000.0,1.5,657000.0,259360.0,8.194892,47.482538,8.194880,47.482537
...,...,...,...,...,...,...,...,...,...,...,...,...
347,2017,Zugersee - Zug ( 2017 ),414.0,250.0,690.0,0.0,681555.0,224660.0,8.514333,47.167874,8.514322,47.167876
348,6723,Zürich - Aussersihl/Zweierplatz ( 6723 ),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
349,6600,Zürich - Schlachthof ( 6600 ),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
350,2014,Zürichsee (Obersee) - Schmerikon ( 2014 ),410.0,1583.0,1333.0,1.1,713710.0,231515.0,8.940092,47.224774,8.940081,47.224777


In [10]:
Stations_BAFU[Stations_BAFU.easting.notnull()].BAFU_ID.values

array([2416, 2312, 2135, 2019, 2016, 2029, 2113, 2085, 2063, 2457, 2030,
       2205, 2031, 2141, 2485, 2232, 2490, 2609, 2299, 2480, 2290, 2170,
       2433, 2137, 2262, 2604, 2333, 2208, 2106, 2122, 2478, 2032, 2043,
       2086, 2023, 2034, 2474, 2447, 2079, 2263, 2436, 2327, 2249, 2270,
       2148, 2370, 2210, 2247, 2053, 2117, 2434, 2409,   70, 2070,  155,
       2155, 2481, 2202, 2305, 2415, 2498,  308, 2308, 2607, 2203, 2082,
       2635, 2276, 2159, 2097, 2387, 2631, 2603,  403, 2403, 2067, 2462,
       2105, 2265, 2418, 2469, 2634, 2487, 2244, 2028, 2027, 2007, 2154,
       2642, 2149, 2057, 2074, 2022, 2150, 2355, 2343, 2484, 2410, 2243,
       2099,  352, 2352, 2372, 2104, 2269, 2125, 2477, 2497, 2109, 2475,
       2368, 2161, 2206, 2369, 2620, 2300, 2420, 2309, 2084, 2386, 2471,
       2126, 2004, 2374, 2371, 2378, 2319, 2304, 2437, 2081,  185, 2185,
       2366,   78, 2078, 2493, 2283, 2430, 2087, 2152, 2018, 2110, 2056,
       2130, 2392, 2615, 2289, 2473, 2602, 2116, 22

In [11]:
Stations_BAFU[Stations_BAFU.easting.isna()].BAFU_ID.values

array([6743, 6572, 6595, 6705, 6755, 6562, 2353, 2489, 6580, 6732, 2384,
       6748, 2349, 6588, 6560, 6583, 6739, 2321, 6578, 6591, 6749, 6704,
       6511, 6592, 2329, 6559, 6607, 2492, 6579, 6738, 6756, 6542, 6567,
       6573, 6558, 6587, 6707, 6709, 6745, 6585, 6598, 6710, 2636, 6741,
       6744, 2071, 2404, 6566, 6535, 6561, 6564, 2101, 2021, 6516, 6590,
       6563, 6575, 6557, 2461, 6574, 6569, 6711, 6758, 6734, 6613, 6601,
       6603, 6577, 6582, 6568, 6713, 6570, 6586, 6735, 6714, 6565, 6604,
       2632, 6715, 6593, 6754, 6612, 6716, 6596, 6599, 6727, 6532, 6576,
       6571, 6746, 6523, 6581, 6510, 6904, 2167, 6550, 9632, 6718, 2463,
        629, 2629, 6543, 6719, 6584, 6720, 6731, 6742, 2156, 6721, 6537,
       6589, 6722, 6594, 6753, 6752, 6723, 6600], dtype=int64)

# Merge test

In [12]:
Stations_ours = pd.read_csv("data_wrangled/info_discharges_gps.csv", index_col="id")
Stations_ours

,Name,BAFU_ID,X,Y,Ave
id,,,,,
1,Allenbach-Adelboden,2232,608710,148300,1.207149
2,Grosstalbach-Isenthal,2276,685500,196050,1.795246
3,"Alpbach-Erstfeld, Bodenberg",2299,688560,185120,1.584263
4,Simme-Oberried / Lenk,2219,602630,141660,2.107648
5,Murg-Wängi,2126,714105,261720,1.835344
...,...,...,...,...,...
64,Reuss-Mellingen,2018,662830,252580,140.732404
65,Rhein-Domat/Ems,2602,753890,189370,117.615329
66,"Rhein-Diepoldsau, Rietbrücke",2473,766280,250360,230.536477


In [13]:
Stations_BAFU.BAFU_ID.values

array([2416, 2312, 2135, 2019, 2016, 2029, 2113, 2085, 2063, 2457, 2030,
       2205, 6743, 2031, 6572, 2141, 2485, 2232, 2490, 2609, 2299, 6595,
       6705, 2480, 2290, 2170, 2433, 6755, 2137, 6562, 2262, 2353, 2489,
       6580, 6732, 2604, 2333, 2208, 2106, 2122, 2478, 2384, 2032, 2043,
       6748, 2349, 2086, 2023, 6588, 6560, 2034, 6583, 6739, 2474, 2447,
       2079, 2321, 2263, 6578, 2436, 6591, 6749, 6704, 6511, 6592, 2329,
       6559, 2327, 6607, 2492, 2249, 2270, 2148, 2370, 2210, 2247, 2053,
       2117, 6579, 2434, 2409,   70, 2070,  155, 2155, 6738, 2481, 6756,
       2202, 6542, 6567, 6573, 2305, 2415, 6558, 2498,  308, 2308, 6587,
       2607, 6707, 2203, 2082, 2635, 2276, 2159, 2097, 6709, 6745, 6585,
       6598, 6710, 2387, 2631, 2636, 6741, 6744, 2603, 2071,  403, 2403,
       2067, 2462, 2105, 2265, 2404, 2418, 2469, 6566, 6535, 6561, 2634,
       2487, 2244, 6564, 2028, 2027, 2007, 2154, 2642, 2149, 2057, 2074,
       2022, 2101, 2021, 6516, 2150, 2355, 2343, 24

In [14]:
Stations_ours.BAFU_ID.values

array([2232, 2276, 2299, 2219, 2126, 2159, 2485, 2122, 2609, 2471, 2434,
       2479, 2307, 2202, 2374, 2112, 2386, 2300, 2409, 2185, 2450, 2603,
       2087, 2488, 2200, 2132, 2498, 2034, 2478, 2480, 2179, 2199, 2151,
       2481, 2155, 2487, 2378, 2070, 2468, 2106, 2247, 2634, 2109, 2469,
       2150, 2303, 2084, 2370, 2270, 2160, 2210, 2372, 2019, 2033, 2056,
       2181, 2044, 2387, 2119, 2215, 2030, 2135, 2110, 2018, 2602, 2473,
       2091, 2289], dtype=int64)

In [15]:
Stations_ours.reset_index(inplace=True)
Discharges_info = Stations_ours.merge(Stations_BAFU, how="left", left_on="BAFU_ID", right_on="BAFU_ID",
                                      suffixes=('', '_BAFU'))
#CSV save to verify all the matching info
#Discharges_info.to_csv("data_wrangled/Discharges_info_merge_debug.csv",index=False)
Discharges_info.set_index("id")

,Name,BAFU_ID,X,Y,Ave,Name_BAFU,Elevation,Surface_catchment,Mean_elevation_catchment,Glaciation_percent,easting,northing,longitude03,latitude03,longitude95,latitude95
id,,,,,,,,,,,,,,,,
1,Allenbach-Adelboden,2232,608710,148300,1.207149,Allenbach - Adelboden ( 2232 ),1297.0,28.8,1863.0,0.0,608710.0,148300.0,7.552069,46.485963,7.552074,46.485963
2,Grosstalbach-Isenthal,2276,685500,196050,1.795246,Grosstalbach - Isenthal ( 2276 ),767.0,43.9,1819.0,6.7,685500.0,196050.0,8.560961,46.910053,8.560952,46.910056
3,"Alpbach-Erstfeld, Bodenberg",2299,688560,185120,1.584263,"Alpbach - Erstfeld, Bodenberg ( 2299 )",1022.0,20.7,2205.0,19.7,688560.0,185120.0,8.599000,46.811344,8.598992,46.811348
4,Simme-Oberried / Lenk,2219,602630,141660,2.107648,Simme - Oberried / Lenk ( 2219 ),1096.0,34.8,2347.0,22.6,602630.0,141660.0,7.472850,46.426283,7.472855,46.426283
5,Murg-Wängi,2126,714105,261720,1.835344,Murg - Wängi ( 2126 ),466.0,80.2,652.0,0.0,714105.0,261720.0,8.953004,47.496328,8.952993,47.496330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,Reuss-Mellingen,2018,662830,252580,140.732404,Reuss - Mellingen ( 2018 ),345.0,3386.0,1259.0,1.8,662830.0,252580.0,8.271283,47.421030,8.271271,47.421030
65,Rhein-Domat/Ems,2602,753890,189370,117.615329,Rhein - Domat/Ems ( 2602 ),575.0,3229.0,2013.0,0.9,753890.0,189370.0,9.456125,46.837667,9.456114,46.837670
66,"Rhein-Diepoldsau, Rietbrücke",2473,766280,250360,230.536477,"Rhein - Diepoldsau, Rietbrücke ( 2473 )",410.0,6299.0,1771.0,0.7,766280.0,250360.0,9.640909,47.383068,9.640905,47.383070
